In [ ]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train')
print(newsgroups_train.target[:10])
newsgroups_test = fetch_20newsgroups(subset='test')
#newsgroups = fetch_20newsgroups()

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vector1 = vectorizer.fit_transform(newsgroups_train.data)
vector_1=vector1

vector2 = vectorizer.transform(newsgroups_train.data)
data=vector2.toarray()
train_label=newsgroups_train.target



[ 7  4  4  1 14 16 13  3  2  4]


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

# def euc_dist(x,y):
#   sq_dist = euclidean_distances(x,y)
#   return sq_dist

def cosine_distance(x,y):
  cosine_distances = cosine_similarity(x,y)
  return cosine_distances

def initialize_centroids(data,k):
  centroids = data[np.random.choice(np.arange(len(data)),k,False)]
  return centroids

def new_centroids(sq_dist, data,k, cosine=False):
  if cosine:
    clusters = np.argmax(sq_dist, axis=0)
  else:
    clusters = np.argmin(sq_dist, axis=0)
  centroids = np.empty(shape=(k,)+data.shape[1:])

  for k in range(k):
    centroids[k] = np.mean(data[clusters == k], axis=0)
  return clusters, centroids

def kmeans(data, k, cosine=False):
  prev_centroids = np.zeros(shape=(k,)+data.shape[1:])
  centroids = initialize_centroids(data,k)
  iteration = 0
  while not np.array_equal(prev_centroids,centroids):
    prev_centroids = centroids.copy()
    iteration+=1

    if cosine:
      distance_matrix = cosine_distance(centroids,data)
    else:
      distance_matrix = euc_dist(centroids,data)
    clusters,centroids = new_centroids(distance_matrix, data,k, cosine)
  return clusters

In [ ]:
clusters= kmeans(data,20,True)

In [ ]:
def clustering_performance(labels, clusters):
  count_per_cluster = []
  n=0
  bins = np.concatenate((np.unique(labels),[np.max(np.unique(labels))+1]),axis=0)
  for cluster in np.unique(clusters):
    #  (unique,counts)=  np.unique(labels[clusters==cluster],return_counts=True)
    #  count_per_cluster.append(counts)
     count_per_cluster.append(np.histogram(labels[clusters==cluster],bins=bins)[0])
  total_true = 0
 
  for cluster_count in count_per_cluster:
    total_true += np.max(cluster_count)
    purity = total_true / clusters.shape[0]
  
  gini_num = 0
  gini_column=[]
  gini_den = 0
  for cluster_count in count_per_cluster:
    total  = np.sum(cluster_count)
    gini_den+=total
    sum = 0
    for count in cluster_count:
      sum +=pow(count/total,2)
    gini_column.append(1-sum)
    gini_num +=((1-sum)*total)
  gini_coefficient = gini_num/gini_den
  return purity, gini_coefficient,gini_column
 
  #return gini_column
print(clustering_performance(train_label,clusters))

(0.2770019444935478, 0.8042579268969261, [0.9287661979402444, 0.7352029247072979, 0.6265942250790736, 0.645055357477951, 0.8596, 0.9143457256878806, 0.07082475965592838, 0.9040156364117339, 0.8117895944637671, 0.547845142439737, 0.7909729448491154, 0.6837422073152883, 0.845679012345679, 0.5422060474787971, 0.9202963830458087, 0.8515820756366179, 0.8198452976253628, 0.7841975308641975, 0.8326124567474049, 0.9397938742923502])
